In [2]:
import os
import shutil
import sys
import io
import zipfile
import gzip
import requests
import warnings
import subprocess

from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.discovery import build
from apiclient import discovery
from httplib2 import Http
import oauth2client
from oauth2client import file, client, tools

# sys.path.insert(1, '../src/data/') # comment out in python script
from load_environmental_variables import *

In [ ]:
print('DOWNLOAD DATA')

In [2]:
# generate relevant data paths
if not os.path.isdir(local_data_path):
    os.mkdir(local_data_path)
if not os.path.isdir(local_data_path + 'figures/'):
    os.mkdir(local_data_path + 'figures/')
if not os.path.isdir(local_data_path + 'raw/'):
    os.mkdir(local_data_path + 'raw/')
    
if not os.path.isdir(local_data_path + 'raw/LB441_fastqs'):
    os.mkdir(local_data_path + 'raw/LB441_fastqs')
if not os.path.isdir(local_data_path + 'raw/LB443_fastqs'):
    os.mkdir(local_data_path + 'raw/LB443_fastqs')   
    
if not os.path.isdir(local_data_path + 'interim/'):
    os.mkdir(local_data_path + 'interim/')
if not os.path.isdir(local_data_path + 'processed/'):
    os.mkdir(local_data_path + 'processed/')


In [23]:
print('Download publically available data, including control and purified Zheng fastqs')
print('Control fastqs')
# control data
file_name = '5k_pbmc_v3_fastqs.tar'
try:
    cmd = 'wget -O ' + local_data_path + 'raw/' + file_name + ' '      
    cmd += 'http://s3-us-west-2.amazonaws.com/10x.files/samples/cell-exp/3.0.2/5k_pbmc_v3/5k_pbmc_v3_fastqs.tar'
    output = subprocess.check_output(cmd, stderr=subprocess.STDOUT, shell=True,universal_newlines=True)
except subprocess.CalledProcessError as exc:
    print("Status : FAIL", exc.returncode, exc.output)

# # untar

os.mkdir(local_data_path + 'raw/' + file_name[:-4])
try:
    cmd = 'tar xf ' + local_data_path + 'raw/' + file_name 
    cmd += ' -C ' + local_data_path + 'raw/' + file_name[:-4] 
    output = subprocess.check_output(cmd, stderr=subprocess.STDOUT, shell=True, universal_newlines=True)
except subprocess.CalledProcessError as exc:
    print("Status : FAIL", exc.returncode, exc.output)

Download publically available control and purified Zheng datasets
Control


In [31]:
print('Zheng')

os.mkdir(local_data_path + 'raw/Zheng_PBMC_fastqs/')
Zfastq_links = {'CD14_Monocytes': 'http://s3-us-west-2.amazonaws.com/10x.files/samples/cell-exp/1.1.0/cd14_monocytes/cd14_monocytes_fastqs.tar',
              'CD19_B_Cells': 'http://s3-us-west-2.amazonaws.com/10x.files/samples/cell-exp/1.1.0/b_cells/b_cells_fastqs.tar',
              'CD34_Cells': 'http://s3-us-west-2.amazonaws.com/10x.files/samples/cell-exp/1.1.0/cd34/cd34_fastqs.tar',
              'CD4_Helper_T_Cells': 'http://s3-us-west-2.amazonaws.com/10x.files/samples/cell-exp/1.1.0/cd4_t_helper/cd4_t_helper_fastqs.tar',
               'CD4_CD25_Regulator_T_Cells': 'http://s3-us-west-2.amazonaws.com/10x.files/samples/cell-exp/1.1.0/regulatory_t/regulatory_t_fastqs.tar',
              'CD4_CD45RA_CD25_Naive_T_Cells': 'http://cf.10xgenomics.com/samples/cell-exp/1.1.0/naive_t/naive_t_fastqs.tar',
              'CD4_CD45RO_Memory_T_Cells': 'http://s3-us-west-2.amazonaws.com/10x.files/samples/cell-exp/1.1.0/memory_t/memory_t_fastqs.tar',
              'CD56_Natural_Killer_Cells': 'http://s3-us-west-2.amazonaws.com/10x.files/samples/cell-exp/1.1.0/cd56_nk/cd56_nk_fastqs.tar',
               'CD8_Cytotoxic_T_Cells': 'http://s3-us-west-2.amazonaws.com/10x.files/samples/cell-exp/1.1.0/cytotoxic_t/cytotoxic_t_fastqs.tar',
              'CD8_CD45RA_Naive_Cytotoxic_T_Cells': 'http://s3-us-west-2.amazonaws.com/10x.files/samples/cell-exp/1.1.0/naive_cytotoxic/naive_cytotoxic_fastqs.tar'}

# k,v = 'CD14_Monocytes', 'http://s3-us-west-2.amazonaws.com/10x.files/samples/cell-exp/1.1.0/cd14_monocytes/cd14_monocytes_fastqs.tar'
counter = 1
for k,v in Zfastq_links.items():
    file_name = v.split('/')[-1]
    print(str(counter) + ': ' + file_name)
    
    try:
        cmd = 'wget -O ' + local_data_path + 'raw/Zheng_PBMC_fastqs/' + file_name + ' '      
        cmd += v
        output = subprocess.check_output(cmd, stderr=subprocess.STDOUT, shell=True,universal_newlines=True)
    except subprocess.CalledProcessError as exc:
        print("Status : FAIL", exc.returncode, exc.output)
    
    # # untar

    os.mkdir(local_data_path + 'raw/Zheng_PBMC_fastqs/' + file_name[:-4])
    try:
        cmd = 'tar xf ' + local_data_path + 'raw/Zheng_PBMC_fastqs/' + file_name 
        cmd += ' -C ' + local_data_path + 'raw/Zheng_PBMC_fastqs/' + file_name[:-4] 
        output = subprocess.check_output(cmd, stderr=subprocess.STDOUT, shell=True, universal_newlines=True)
    except subprocess.CalledProcessError as exc:
        print("Status : FAIL", exc.returncode, exc.output)
    
    counter += 1

# # Zheng datasets for classficiation

# # Attempt 1: using the github scripts
# try:
#     cmd = 'git clone https://github.com/10XGenomics/single-cell-3prime-paper.git ' + local_data_path + 'raw/'
#     output = subprocess.check_output(cmd, stderr=subprocess.STDOUT, shell=True, 
#                                      timeout=3,universal_newlines=True)
#     except subprocess.CalledProcessError as exc:
#         print("Status : FAIL", exc.returncode, exc.output)

# try:
#     cmd = 'wget -O ' + local_data_path + 'raw/ '
#     cmd += 'http://s3-us-west-2.amazonaws.com/10x.files/samples/cell/pbmc68k_rds/all_pure_pbmc_data.rds'
#     output = subprocess.check_output(cmd, stderr=subprocess.STDOUT, shell=True, 
#                                      timeout=3,universal_newlines=True)
#     except subprocess.CalledProcessError as exc:
#         print("Status : FAIL", exc.returncode, exc.output)


In [ ]:
print('Single cell markers')

file_name = 'Single_cell_markers.txt'
try:
    cmd = 'wget -O ' + local_data_path + 'raw/' + file_name + ' '      
    cmd += 'http://biocc.hrbmu.edu.cn/CellMarker/download/Single_cell_markers.txt'
    output = subprocess.check_output(cmd, stderr=subprocess.STDOUT, shell=True,universal_newlines=True)
except subprocess.CalledProcessError as exc:
    print("Status : FAIL", exc.returncode, exc.output)

In [26]:
print('Download LB441 and LB443 from Google Drive')
# this will only work if you have a client_id.json in root_path

try:
    obj = lambda: None
    lmao = {"auth_host_name":'localhost', 'noauth_local_webserver':'store_true', 'auth_host_port':[8080, 8090], 'logging_level':'ERROR'}
    for k, v in lmao.items():
        setattr(obj, k, v)

    # authorization boilerplate code
    SCOPES = 'https://www.googleapis.com/auth/drive.readonly'
    store = file.Storage(root_path + 'token.json')
    creds = store.get()
    # The following will give you a link if token.json does not exist, the link allows the user to give this app permission
    if not creds or creds.invalid:
        flow = client.flow_from_clientsecrets(root_path + 'client_id.json', SCOPES)
        creds = tools.run_flow(flow, store, obj)
except:
    raise ValueError('Have you created a client_id.json?')
    raise ValueError ('Have you created a .env in root directory with appropriate variables?')
    raise ValueError('Current cell may first need to be run in jupyter notebook (see projects/notebooks/download_dataset.ipynb) to create a token.json')
    
    
service = discovery.build('drive', 'v2', http=creds.authorize(Http()))
# get ID and names of all files in a specified folder (by the folder_id in .env file)
children = service.children().list(folderId=folder_id).execute()
children_ids = [child['id'] for child in children.get('items', [])]
file_names = [service.files().get(fileId=file_id).execute()['title'] for file_id in children_ids]

# parse
files = list(zip(children_ids, file_names))
files = [file for file in files if 'fastq.gz' in file[1]] # COMMENT OUT IF REUSING IN OTHER PROJECTS


# download files from drive

DRIVE = discovery.build('drive', 'v3', http=creds.authorize(Http()))
count = 1
for file in files:
    file_id = file[0]
    file_name = file[1]
    print(str(count) + ' of ' + str(len(files)) + ' files: ' + file_name)
    # if you get the shareable link, the link contains this id, replace the file_id below
    request = DRIVE.files().get_media(fileId=file_id)

    # replace the filename and extension in the first field below
    fh = io.FileIO(file_name, mode='w')
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print("Download %d%%." % int(status.progress() * 100))

    # move files to raw data path
    if 'LB441' in file_name:
        str_ = 'mv ' + os.getcwd() + '/' + file_name + ' ' + local_data_path + 'raw/LB441_fastqs/' + file_name
    elif 'LB443' in file_name:
        str_ = 'mv ' + os.getcwd() + '/' + file_name + ' ' + local_data_path + 'raw/LB443_fastqs/' + file_name
    os.system(str_)
    
    count += 1

Download LB441 and LB443 from Google Drive


ValueError: Have you created a client_id.json?

# Part 2

In [4]:
# # download the second 10k pbmc dataset
data_path = '/data3/hratch/STAT4/'
hl = 'https://s3-us-west-2.amazonaws.com/10x.files/samples/cell-exp/3.0.0/pbmc_10k_v3/pbmc_10k_v3_fastqs.tar'

# generate relevant data paths
if not os.path.isdir(data_path):
    os.mkdir(data_path)
for dir_ in ['figures/', 'raw/', 'processed/', 'interim/']:
    if not os.path.isdir(data_path + dir_):
        os.mkdir(data_path + dir_)
for dir_ in ['LB441_fastqs', 'LB443_fastqs']:
    if not os.path.isdir(data_path + 'raw/' + dir_):
        os.mkdir(data_path + 'raw/' +  dir_)

cmd = 'wget -P ' + data_path + 'raw/10k_pbmc_control/ ' + hl
os.system(cmd)

cmd = 'tar xvf ' + data_path + 'raw/10k_pbmc_control/pbmc_10k_v3_fastqs.tar '
cmd += ' -C ' + data_path + 'raw/10k_pbmc_control/'
os.system(cmd)

In [29]:
# print('Download LB441 and LB443 from Google Drive')
# # this will only work if you have a client_id.json in root_path

# try:
#     obj = lambda: None
#     lmao = {"auth_host_name":'localhost', 'noauth_local_webserver':'store_true', 'auth_host_port':[8080, 8090], 'logging_level':'ERROR'}
#     for k, v in lmao.items():
#         setattr(obj, k, v)

#     # authorization boilerplate code
#     SCOPES = 'https://www.googleapis.com/auth/drive.readonly'
#     store = file.Storage(root_path + 'token.json')
#     creds = store.get()
#     # The following will give you a link if token.json does not exist, the link allows the user to give this app permission
#     if not creds or creds.invalid:
#         flow = client.flow_from_clientsecrets(root_path + 'client_id.json', SCOPES)
#         creds = tools.run_flow(flow, store, obj)
# except:
#     raise ValueError('Have you created a client_id.json?')
#     raise ValueError ('Have you created a .env in root directory with appropriate variables?')
#     raise ValueError('Current cell may first need to be run in jupyter notebook (see projects/notebooks/download_dataset.ipynb) to create a token.json')
    
    
# service = discovery.build('drive', 'v2', http=creds.authorize(Http()))
# # get ID and names of all files in a specified folder (by the folder_id in .env file)
# children = service.children().list(folderId=folder_id).execute()
# children_ids = [child['id'] for child in children.get('items', [])]
# file_names = [service.files().get(fileId=file_id).execute()['title'] for file_id in children_ids]

# # parse
# files = list(zip(children_ids, file_names))
# files = [file for file in files if 'fastq.gz' in file[1]] # COMMENT OUT IF REUSING IN OTHER PROJECTS


# # download files from drive

# DRIVE = discovery.build('drive', 'v3', http=creds.authorize(Http()))
# count = 1
# for file in files:
#     file_id = file[0]
#     file_name = file[1]
#     print(str(count) + ' of ' + str(len(files)) + ' files: ' + file_name)
#     # if you get the shareable link, the link contains this id, replace the file_id below
#     request = DRIVE.files().get_media(fileId=file_id)

#     # replace the filename and extension in the first field below
#     fh = io.FileIO(file_name, mode='w')
#     downloader = MediaIoBaseDownload(fh, request)
#     done = False
#     while done is False:
#         status, done = downloader.next_chunk()
#         print("Download %d%%." % int(status.progress() * 100))

#     # move files to raw data path
#     if 'LB441' in file_name:
#         str_ = 'mv ' + os.getcwd() + '/' + file_name + ' ' + data_path + 'raw/LB441_fastqs/' + file_name
#     elif 'LB443' in file_name:
#         str_ = 'mv ' + os.getcwd() + '/' + file_name + ' ' + data_path + 'raw/LB443_fastqs/' + file_name
#     os.system(str_)
    
#     count += 1